In [3]:
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

drive.mount('/content/drive')

input_folder = '/content/drive/MyDrive/Project Folder/three_glasses_later/3_glasses_later'
output_folder = '/content/drive/MyDrive/processed_images'
os.makedirs(output_folder, exist_ok=True)


classification_folders = ["sober", "one_glass", "three_glasses"]
for folder in classification_folders:
    os.makedirs(os.path.join(output_folder, folder), exist_ok=True)

# Initialize a summary file
summary_data = []

# Function to save split images with the correct label
def save_split_image(image, label, person_id):
    # Simplify the filename to just the person ID
    output_path = os.path.join(output_folder, label, f"{person_id}.jpg")
    cv2.imwrite(output_path, image)

    # Add entry to summary data
    summary_data.append({"Person": f"person_{person_id}", "Label": label, "Image": output_path})
    print(f"Saved {output_path} as {label}")

# Function to display split images for verification
def display_split_images(images, labels, person_id):
    plt.figure(figsize=(10, 5))
    for i, (image, label) in enumerate(zip(images, labels)):
        plt.subplot(1, 3, i + 1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f"{label} (Person {person_id})")
        plt.axis("off")
    plt.show()

# Process each 2x2 grid image
person_id = 1
for filename in os.listdir(input_folder):
    if filename.endswith((".png", ".jpg")):
        # Load the grid image
        image_path = os.path.join(input_folder, filename)
        grid_image = cv2.imread(image_path)

        # Get the dimensions of the grid image
        grid_h, grid_w, _ = grid_image.shape

        # Split the grid into three relevant individual images
        sober_image = grid_image[0:grid_h // 2, 0:grid_w // 2]  # Top-left
        one_glass_image = grid_image[0:grid_h // 2, grid_w // 2:grid_w]  # Top-right
        three_glasses_image = grid_image[grid_h // 2:grid_h, grid_w // 2:grid_w]  # Bottom-right

        # Organize split images and labels
        split_images = [sober_image, one_glass_image, three_glasses_image]
        labels = ["sober", "one_glass", "three_glasses"]

        # Display split images for verification
        display_split_images(split_images, labels, person_id)

        # Save each split image with the corresponding label
        for label, image in zip(labels, split_images):
            save_split_image(image, label, person_id)

        # Increment person ID for the next grid
        person_id += 1

# Save the summary as a CSV
summary_df = pd.DataFrame(summary_data)
summary_csv_path = os.path.join(output_folder, "classification_summary.csv")
summary_df.to_csv(summary_csv_path, index=False)

print(f"Splitting complete. Summary saved to {summary_csv_path}")

Output hidden; open in https://colab.research.google.com to view.